# 0 - Set-Up

### Environment Set Up 

In [ ]:
# Install required packages
!pip install -qq -r ../requirements.txt

# Set up root directory
import sys

REL_PATH_TO_ROOT = "../"

sys.path.insert(0,REL_PATH_TO_ROOT)

from src.utils import get_root_dir, test_root_dir
from local_variables import ROOT_DIR

test_root_dir(REL_PATH_TO_ROOT)

In [ ]:
# Get project questions
from questions.questions import questions as QQQQ

### Import Packages

In [ ]:
# Import project specific packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from scipy.stats import mannwhitneyu

### Load Data

In [ ]:
# Relevant files for AirBnB
files_of_interest = ["calendar.csv","listings.csv"]
years = ["2023","2024"]

In [ ]:
# Concatenate for 2023 and 2024
cal_df = pd.concat([pd.read_csv(f"{get_root_dir()}/data/{year}_tokyo/calendar.csv") for year in years],ignore_index=True)

In [ ]:
# Concatenate for 2023 and 2024
list_df = pd.concat([pd.read_csv(f"{get_root_dir()}/data/{year}_tokyo/listings.csv") for year in years],ignore_index=True)

In [ ]:
first_bloom = pd.read_csv(f"{get_root_dir()}/data/cherry_blossom_dates/sakura_first_bloom_dates.csv")
first_bloom.columns = first_bloom.columns.str.lower().str.replace(" ","_")

In [ ]:
full_bloom = pd.read_csv(f"{get_root_dir()}/data/cherry_blossom_dates/sakura_full_bloom_dates.csv")
full_bloom.columns = full_bloom.columns.str.lower().str.replace(" ","_")

In [ ]:
# Print out questions
def get_question(question_num):
    print(QQQQ[question_num])

for i in range(1,5):
    get_question(str(i))

# 1 - Initial EDA

## 1.1 - High-Level View - Calendar

### 1.1.1 - Data Structure

In [ ]:
cal_df.head()

In [ ]:
print("Each row corresponds to one data and one property based on the listing id")

In [ ]:
#Number of unique properties
cal_df["listing_id"].nunique()

### 1.1.2 - Date Range

In [ ]:
#Time range
cal_df["date_dt"] = pd.to_datetime(cal_df["date"])
cal_df["date_dt"].dt.to_period('M').value_counts().sort_index().plot(kind='line')

In [ ]:
cal_df.shape[0]

In [ ]:
# Jump in counts for 2024-07, are there duplicates?
dropped_dups = cal_df.drop_duplicates()

In [ ]:
dropped_dups.shape[0]

In [ ]:
print("No duplicates found")

### 1.1.3 - Price Distribution

In [ ]:
# Turn price string into a number
cal_df["price_num"] = cal_df["price"].str.replace("$","")
cal_df["price_num"] = cal_df["price_num"].str.replace(",","")
cal_df["price_num"] = cal_df["price_num"].astype(float)

In [ ]:
# Plot distribution (use log scale due to wide variation of values)
sns.histplot(data=cal_df,x="price_num",log_scale=True,bins=15)

### 1.1.4 - Average Prices Over Time

In [ ]:
def average_x_time_plot(df, datefield, x, show_percentiles=False, percentile=0.95, group_var=None, log_scale=True):
    """
    Plots the average of a specified column (`x`) over time, with options for percentile bounds 
    and grouping by an additional variable.

    Parameters:
    ----------
    df : pandas.DataFrame
        The input DataFrame containing the data to be plotted.
    datefield : str
        The name of the column representing dates.
    x : str
        The column for which averages are computed and plotted.
    show_percentiles : bool, optional, default=False
        Whether to display percentile bounds on the plot. 
        If True, shaded areas showing percentile bounds are added.
    percentile : float, optional, default=0.95
        The desired percentile range to display if `show_percentiles` is True. 
        For example, 0.95 corresponds to a 95% percentile range.
    group_var : str or None, optional, default=None
        An optional column for grouping the data. Separate lines and percentile bounds are plotted 
        for each unique value in this column if provided.
    log_scale : bool, optional, default=True
        Whether to apply a natural logarithm transformation to the averages and bounds 
        before plotting.

    Returns:
    -------
    None
        The function generates and displays the plot.

    Notes:
    ------
    - If `log_scale` is True, the y-axis will represent the log-transformed values of `x`.
    - When `show_percentiles` is True and `group_var` is provided, separate shaded areas 
      representing the percentile range are plotted for each group.
    - The function uses Seaborn for line plotting and Matplotlib for additional styling.

    Example:
    --------
    average_x_time_plot(
        df=my_data, 
        datefield="date", 
        x="value", 
        show_percentiles=True, 
        percentile=0.9, 
        group_var="category", 
        log_scale=False
    )
    """

    # Copy dataframe to avoid making changes
    work_df = df.copy(deep=False)

    # Convert data column to DT format
    work_df[datefield] = pd.to_datetime(work_df[datefield])

    # Set grouping variables depending on inputs
    if group_var == None:
        group_fields = datefield
    else:
        group_fields = [datefield,group_var]

    # Get the summary stats depending on whether upper or lower bounds were wanted
    if show_percentiles:
        perc = 1-percentile
        summary_stats = work_df.groupby(group_fields)[x].agg(avg='mean',lower_bound=lambda x: x.quantile(perc/2),
    upper_bound=lambda x: x.quantile(1-perc/2)).reset_index()
    else:
        summary_stats = work_df.groupby(group_fields)[x].agg(avg='mean').reset_index()

    # Create necessary transformed variables depending on log_scale argument 
    if log_scale:
        summary_stats[f"log_{x}"] = np.log(summary_stats['avg'])
        if show_percentiles:
            summary_stats['log_lower_bound'] = np.log(summary_stats['lower_bound'])
            summary_stats['log_upper_bound'] = np.log(summary_stats['upper_bound'])

    # Create plot
    plt.figure(figsize=(12, 6))

    # Create a palette to keep colouring of bounds consistent with the line plot
    sns_palette = sns.color_palette(n_colors=summary_stats[group_var].nunique() if group_var else 1)

    # Plot lineplot of date against (log) of average x over time
    sns.lineplot(data=summary_stats, x=summary_stats[datefield],y=f"log_{x}" if log_scale else 'avg',hue=group_var, linewidth=2,palette=sns_palette)

    # Create bound plots if requested
    if show_percentiles and group_var:
        for color, (group, group_data) in zip(sns_palette, summary_stats.groupby(group_var)):
            plt.fill_between(
                group_data[datefield],
                group_data['log_lower_bound'] if log_scale else group_data['lower_bound'],
                group_data['log_upper_bound'] if log_scale else group_data['upper_bound'],
                alpha=0.2,
                color=color,
                label=f'{group} {100 * percentile}% Percentile Range'
            )

    elif show_percentiles:  # For no group_var, single fill_between
        plt.fill_between(
            summary_stats[datefield],
            summary_stats['log_lower_bound'] if log_scale else summary_stats['lower_bound'],
            summary_stats['log_upper_bound'] if log_scale else summary_stats['upper_bound'],
            alpha=0.3,
            color=sns_palette[0],
            label=f'{100 * percentile}% Percentile Range'
        )
    
    plt.xlabel('Date')
    plt.ylabel(f'Log ')
    plt.title(f'Average {'Log ' if log_scale else ""}{x.capitalize()} Over Time with {100*percentile}% Percentile Bounds')
    plt.legend()
    plt.show()

In [ ]:
# Plot average log price over time
average_x_time_plot(cal_df,"date","price_num",show_percentiles=True)

In [ ]:
print("Huge variability in the price at a given time, what determines the price?")

In [ ]:
# Look at availability for price 
average_x_time_plot(cal_df,"date","price_num",show_percentiles=True,group_var="available")

In [ ]:
print("Doesn't seem to make a huge difference, although unoccupied properties tend to be priced lower")

In [ ]:
# Look at availability for price 
plot_1_df = cal_df[cal_df["maximum_nights"] <= 31]
plot_2_df = cal_df[~(cal_df["maximum_nights"] <= 31)]

In [ ]:
plot_1_df['maximum_nights_group'] = pd.qcut(plot_1_df['maximum_nights'], q=4,duplicates="drop")
average_x_time_plot(plot_1_df,"date","price_num",show_percentiles=False,group_var="maximum_nights_group")

In [ ]:
print("Seems that smaller maximum stays are generally more expensive, while maximum stays after a certain point are more comparable. Huge drop and rebound in 2024-07 may correspond to the large depreciation of the Yen (-8% agains the dollar) which occurred early July")

In [ ]:
plot_2_df['maximum_nights_group'] = pd.qcut(plot_2_df['maximum_nights'], q=5,duplicates="drop")
average_x_time_plot(plot_2_df,"date","price_num",show_percentiles=False,group_var="maximum_nights_group")

In [ ]:
print("Properties with a likely placeholder maximum stay seem to have a precipitous decline for projected prices")

In [ ]:
cal_df.columns

In [ ]:
plot_1_df = cal_df[cal_df["minimum_nights"] <= 15]
plot_1_df['minimum_nights_group'] = pd.qcut(plot_1_df['minimum_nights'], q=3,duplicates="drop")
average_x_time_plot(plot_1_df,"date","price_num",show_percentiles=False,group_var="minimum_nights_group")

In [ ]:
print("Seems to be a difference but not huge in size")

In [ ]:
plot_2_df = cal_df[~(cal_df["minimum_nights"] <= 15)]
plot_2_df['minimum_nights_group'] = pd.qcut(plot_2_df['minimum_nights'], q=3,duplicates="drop")
average_x_time_plot(plot_2_df,"date","price_num",show_percentiles=False,group_var="minimum_nights_group")

In [ ]:
print("Seems to be a huge difference for before 07")

## 1.2 - Listings

In [ ]:
# View columns in the listings dataframe which provides more information on individual hosts 
list_df.columns

In [ ]:
list_df.head()

In [ ]:
print("Unique to id and last_scraped level")

### 1.2.1 - Time Structure in Dataframe

In [ ]:
# Get a list of ids
test_ids = list_df["id"].head().values

In [ ]:
# See how things change over time
list_df[list_df["id"].isin(test_ids)].sort_values(by="id")

In [ ]:
# How many listings have data for both years?
list_df["id"].value_counts().reset_index()["count"].value_counts(normalize=True)

In [ ]:
print("Around 50% only appear for one year")

In [ ]:
# Create average price table 
average_prices = cal_df[["listing_id","price_num"]].groupby(by="listing_id",as_index=False).agg("mean")

### 1.2.2 - Review Data

In [ ]:
# Plot histogram for review rating
list_df["review_scores_rating"].hist(bins=5)

In [ ]:
# Take log to spread out distribution a bit more, add offset for reviews of 0
np.log(list_df["review_scores_rating"]+0.1).hist(bins=10)

In [ ]:
list_df["review_scores_rating"].isna().mean()

In [ ]:
stars_and_prices = pd.merge(left=average_prices,right=list_df[["id","review_scores_rating"]].rename(columns={"id":"listing_id"}),how="inner",on="listing_id")

In [ ]:
sns.scatterplot(data=stars_and_prices,x="review_scores_rating",y="price_num")

In [ ]:
# Let's remove outliers on price

lower_quantile = 0.05
upper_quantile = 0.95

# Calculate the lower and upper bounds
price_lower = stars_and_prices['price_num'].quantile(lower_quantile)
price_upper = stars_and_prices['price_num'].quantile(upper_quantile)

# Filter the data to remove outliers
filtered_data = stars_and_prices[
    (stars_and_prices['price_num'] >= price_lower) &
    (stars_and_prices['price_num'] <= price_upper)
]


sns.scatterplot(data=filtered_data,x="review_scores_rating",y="price_num")


In [ ]:
print("Still quite a lot of variability, hard to spot a trend from the visualisation")

### 1.2.3 - Room Type

In [ ]:
roomgen_and_price = pd.merge(left=average_prices,right=list_df[["id","room_type"]].rename(columns={"id":"listing_id"}),how="inner",on="listing_id")

### 1.2.4 - Number of Rooms

In [ ]:
price_room_df = pd.merge(left=average_prices,right=list_df[["id","bedrooms"]].rename(columns={"id":"listing_id"}),how="inner",on="listing_id")

In [ ]:
perc=0.05
price_by_room_df = price_room_df[["bedrooms","price_num"]].groupby(by="bedrooms",as_index=True)["price_num"].agg(avg_price="mean",lower_bound=lambda x: x.quantile(perc/2),
    upper_bound=lambda x: x.quantile(1-perc/2)).reset_index()

In [ ]:
price_by_room_df.head()

In [ ]:
sns.barplot(data=price_by_room_df.sort_values(by="bedrooms",ascending=True),x="bedrooms",y="avg_price")
sns.barplot(data=price_by_room_df.sort_values(by="bedrooms",ascending=True),x="bedrooms",y="lower_bound",color='r',alpha=0.3)
sns.barplot(data=price_by_room_df.sort_values(by="bedrooms",ascending=True),x="bedrooms",y="upper_bound",color='g',alpha=0.3)

In [ ]:
print("Bedrooms seems to have some correlation to price for less than 5 bedrooms, perhaps 6 bedrooms represent hostels etc.")

In [ ]:
list_df["bedrooms"].value_counts().reset_index().sort_values(by="bedrooms",ascending=True)

### 1.2.5 - Neighbourhood

In [ ]:
list_df["neighbourhood"].nunique()

In [ ]:
list_df["neighbourhood_cleansed"].nunique()

In [ ]:
neighbourhood_price_df = pd.merge(left=average_prices,right=list_df[["id","neighbourhood_cleansed"]].rename(columns={"id":"listing_id"}),how="inner",on="listing_id")

In [ ]:
price_by_neigh_df = neighbourhood_price_df.groupby("neighbourhood_cleansed",as_index=False).agg("mean")

In [ ]:
sns.barplot(data=price_by_neigh_df.sort_values(by="price_num",ascending=False),x="neighbourhood_cleansed",y="price_num",hue="neighbourhood_cleansed")

In [ ]:
print("Neighbourhood seems to have a big impact on price")

### 1.2.6 - All Numerical Variables

In [ ]:
types_df = list_df.dtypes.reset_index()
types_df.columns = ["col","type"]
types_df["type"] = types_df["type"].astype(str)

In [ ]:
num_vars = types_df[types_df["type"].apply(lambda x : True if ("int" in x or "float" in x) else False)]

In [ ]:
num_vars["na_rate"] = num_vars["col"].apply(lambda x : list_df[x].isna().mean())

In [ ]:
sns.barplot(data=num_vars.sort_values(by="na_rate",ascending=False),y="na_rate",x="col")
plt.xticks(rotation=90)

In [ ]:
num_var_cols = num_vars["col"].values

In [ ]:
corr_df = pd.merge(left=list_df[num_var_cols],right=average_prices.rename(columns={"listing_id" : "id"}),how="inner",on="id")
corr_df.drop(columns=["id"],inplace=True)

In [ ]:
# Compute correlations
correlations = corr_df.corr()['price_num'].drop('price_num')  # Exclude price

# Plot bar chart
correlations.plot(kind='bar', title='Correlations with Price')
plt.xlabel('Variables')
plt.ylabel('Correlation')
plt.show()


### 1.2.6 - All Categorical Variables

In [ ]:
# Reverse of the num_vars condition
cat_vars = types_df[types_df["type"].apply(lambda x : False if ("int" in x or "float" in x) else True)]

In [ ]:
# Example values
for col in cat_vars["col"].values:
    print(f"""
    {col}
    {list_df[col].sample(n=1,random_state=42).iloc[0]}
    """)

In [ ]:
# Which fields are redundant i.e. links or names, non-comparable data or free-text?
redundant_fields = ["listing_url",
                    "name"
                    "source",
                    "description",
                    "host_url",
                    "host_name",
                    "host_about",
                    "host_thumbnail_url",
                    "host_picture_url",
                    "license",
                    "picture_url",
                    "name",
                    "neighborhood_overview",
                    "last_scraped",
                   "first_review",
                   "last_review",
                   "calendar_last_scraped",
                    "host_since",
                    "price",
                    "amenities"
                   ]

In [ ]:
cat_cols = cat_vars[cat_vars["col"].apply(lambda x : True if x not in redundant_fields else False)]

In [ ]:
cat_cols

In [ ]:
cat_df = pd.get_dummies(list_df[["price"] + cat_cols["col"].to_list()], columns=cat_cols["col"].to_list(),drop_first=True)

In [ ]:
cat_df

### 1.2.7 - Running A Basic Model

In [ ]:
# Remove NaNs for modelling
model_df = cat_df.dropna(subset="price")

In [ ]:
model_df["price_transformed"] = model_df["price"].apply(lambda x : float(x.replace(",","").replace("$","")))

In [ ]:
X = model_df[[col for col in model_df.columns if col not in ["price","price_transformed"]]]

In [ ]:
y= model_df["price_transformed"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")

In [ ]:
print("Model performs terribly!")

## 1.3 - Cherry Blossom Data

### 1.3.1 - High Level View

In [ ]:
first_bloom.head()

In [ ]:
full_bloom.head()

In [ ]:
# Locations covered
first_bloom["site_name"].unique()

In [ ]:
## Compare to neighbourhoods 
neighbourhoods = list_df["neighbourhood_cleansed"].unique()
bloom_sites = first_bloom["site_name"].unique()


# Make into single strings
neighbourhoods = [neighbourhood.split()[0] for neighbourhood in list_df["neighbourhood_cleansed"].unique()]
bloom_sites = [site.split()[0] for site in first_bloom["site_name"].unique()]

# Find crossover 
intersection = set(neighbourhoods).intersection(set(bloom_sites))
print(intersection)

In [ ]:
# Only the 'Tokyo Japan' area covers our AirBnB data
tokyo_first_bloom = first_bloom[first_bloom["site_name"] == "Tokyo Japan"]
tokyo_full_bloom = full_bloom[full_bloom["site_name"] == "Tokyo Japan"]

In [ ]:
tokyo_first_bloom

In [ ]:
tokyo_full_bloom

In [ ]:
print("Does not cover out date range, can we find the dates elswhere?")

In [ ]:
print("""Using :{https://www.jrailpass.com/blog/japan-cherry-blossom-forecast}

first bloom is 2024-03-29 and full bloom is 2024-04-04
""")

In [ ]:
near_to_bloom_cal_df = cal_df[(pd.to_datetime(cal_df["date"]) < '2024-05-01') & (pd.to_datetime(cal_df["date"]) > '2024-02-01')]

In [ ]:
# Look at availability for price 
average_x_time_plot(near_to_bloom_cal_df,"date","price_num")

In [ ]:
print("Seems to be a change, can we check using the Mann-Whitney U test")

# 2 - Answering Questions

## 2.1 - Q1

In [ ]:
get_question("1")

### 2.1.1 - Try A Model

In [ ]:
# Take smaller sample to minimise data set size 
print(list_df["id"].nunique())

In [ ]:
# Take a stratified sample

# Define sample fraction (e.g., 50% from each group)
sample_fraction = 0.2

# Perform stratified sampling
stratified_sample = list_df.groupby('neighbourhood_cleansed', group_keys=False).apply(lambda x: x.sample(frac=sample_fraction,random_state=42))
ids = list(stratified_sample["id"].unique())
print(f"Now there are {len(ids)} ids")

In [ ]:
# Get neighbourhood information
q1_df = pd.merge(left=cal_df[cal_df["listing_id"].isin(ids)].rename(columns={"listing_id" : "id"}),right=list_df[list_df["id"].isin(ids)][["id","neighbourhood_cleansed"]],how="inner",on="id")

In [ ]:
# Turn neighbourhood into categorical columns 
q1_df = pd.get_dummies(q1_df,columns=["neighbourhood_cleansed"],drop_first=True,dtype=int)

In [ ]:
# Remove nans for prediction variable
q1_df = q1_df.dropna(subset="price_num")

In [ ]:
q1_df.head()

In [ ]:
q1_df.shape

In [ ]:
# Remove prices of 0 before logging 
q1_df = q1_df[q1_df["price_num"] > 0]

In [ ]:
q1_df.shape

In [ ]:
X = q1_df[[col for col in q1_df.columns if "neighbourhood_cleansed" in col]]

In [ ]:
y = np.log(q1_df["price_num"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
neighbourhood_model = LinearRegression()
neighbourhood_model.fit(X_train, y_train)

In [ ]:
y_pred = neighbourhood_model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")

In [ ]:
print("It doesn't predict it very well, what about if we just look at average price?")

### 2.2.2 - Simplified Model

In [ ]:
q1_avg_df = list_df[["neighbourhood_cleansed","price"]]

In [ ]:
q1_avg_df["price_num"] = q1_avg_df["price"].apply(lambda x : str(x).replace(",","").replace("$","")).astype(float)

In [ ]:
q1_avg_df = q1_avg_df.dropna(subset="price_num")

In [ ]:
q1_avg_df = q1_avg_df[q1_avg_df["price_num"] > 0]

In [ ]:
q1_avg_df["log_price"] = np.log(q1_avg_df["price_num"])

In [ ]:
q1_avg_df = pd.get_dummies(q1_avg_df,columns=["neighbourhood_cleansed"],drop_first=True,dtype=int).dropna(subset="log_price")

In [ ]:
q1_avg_df.head()

In [ ]:
X = q1_avg_df[[col for col in q1_avg_df.columns if "neighbourhood_cleansed" in col]]
y = q1_avg_df["log_price"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
simpler_neighbourhood_model = LinearRegression()
simpler_neighbourhood_model.fit(X_train, y_train)

In [ ]:
y_pred = simpler_neighbourhood_model.predict(X_test)

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R²: {r2}")

In [ ]:
print("Still performs poorly, other features are clearly important")

## 2.2 - Q2

### 2.2.1 - Performing Significance Test